# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [22]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

630


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [23]:
# for index, city in enumerate(cities):

#     base_url = "http://api.openweathermap.org/data/2.5/weather?"

#     weather_url = f'{base_url}q={city}&appid={weather_api_key}'
#     response = requests.get(weather_url).json()
# #     pprint(response)

In [36]:
cities_df = pd.DataFrame({"city_name": [],
                        "latitude": [],
                        "longitude": [],
                        "max temp": [],
                        "humidity": [],
                        "cloudiness": [],
                        "wind speed": [],
                        "country": [],
                        "date": []})

for index, city in enumerate(cities):
    
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    weather_url = f'{base_url}q={city}&appid={weather_api_key}'
    response = requests.get(weather_url).json()

    try:
        print(f"Processing Record {index} | {city}.")
        cities_df.loc[index, 'city_name'] = response['name']
        cities_df.loc[index, 'latitude'] = response['coord']['lat']
        cities_df.loc[index, 'longitude'] = response['coord']['lon']
        cities_df.loc[index, 'max temp'] = response['main']['temp_max']
        cities_df.loc[index, 'humidity'] = response['main']['humidity']
        cities_df.loc[index, 'cloudiness'] = response['clouds']['all']
        cities_df.loc[index, 'wind speed'] = response['wind']['speed']
        cities_df.loc[index, 'country'] = response['sys']['country']
        cities_df.loc[index, 'date'] = response['dt']

    except (KeyError, NameError):
        print("City not found. Skipping...")

Processing Record 0 | rikitea.
Processing Record 1 | atuona.
Processing Record 2 | hayden.
Processing Record 3 | busselton.
Processing Record 4 | tigil.
Processing Record 5 | jamestown.
Processing Record 6 | riyadh.
Processing Record 7 | cape town.
Processing Record 8 | touros.
Processing Record 9 | kavieng.
Processing Record 10 | illoqqortoormiut.
City not found. Skipping...
Processing Record 11 | carnarvon.
Processing Record 12 | port lincoln.
Processing Record 13 | solwezi.
Processing Record 14 | awbari.
Processing Record 15 | abilene.
Processing Record 16 | beyneu.
Processing Record 17 | hami.
Processing Record 18 | bambous virieux.
Processing Record 19 | guerrero negro.
Processing Record 20 | san carlos de bariloche.
Processing Record 21 | bengkulu.
Processing Record 22 | kahului.
Processing Record 23 | assiniboia.
Processing Record 24 | paamiut.
Processing Record 25 | skjaerhollen.
City not found. Skipping...
Processing Record 26 | upernavik.
Processing Record 27 | hilo.
Processi

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [37]:
cities_df.head()

,city_name,latitude,longitude,max temp,humidity,cloudiness,wind speed,country,date
0,Rikitea,-23.1203,-134.9692,299.30,74.0,5.0,9.04,PF,1.616818e+09
1,Atuona,-9.8000,-139.0333,300.39,75.0,57.0,8.61,PF,1.616818e+09
2,Hayden,47.7660,-116.7866,281.48,57.0,90.0,2.57,US,1.616818e+09
3,Busselton,-33.6500,115.3333,297.04,50.0,25.0,0.89,AU,1.616818e+09
4,Tigil',57.8000,158.6667,266.46,76.0,87.0,4.92,RU,1.616818e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression